In [1]:
# !pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

#Build in Tools / Agents function

## 1. DuckDuckGo Search

Simply link with real time tool with LLM and user will know latest News.

In [3]:
!pip install -U ddgs
from langchain_community.tools import DuckDuckGoSearchRun
search_from=DuckDuckGoSearchRun()
response=search_from.invoke("who is the president of france")
print(response)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 13.9 MB/s eta 0:00:00


As the presidency is the supreme magistracy of the country, the position is the highest office in France . The presidency of France was first publicly proposed during the July Revolution of 1830, when it was offered to the Marquis de Lafayette. Find out all the news of the President Emmanuel Macron on the official website and discover our pages on the history of the French Republic.Yesterday, 150th anniversary of the Republic. The Presidents since 1848. The outgoing president won 27.8% of the first-round vote, while the leader of the far-right nationalist Rassemblement National (National Rally) managed 23.1%. Find all the latest articles and watch TV shows, reports and podcasts related to François Fillon on France 24.'Penelopegate': Francois Fillon, centre. and his wife Penelope were ensnared in a graft scandal that torpedoed his bid for the French presidency . Emmanuel Macron has been re-elected as President of France . What does Macron's victory mean for the voters, Europe, and India

## 2. Shell tool
we can run command line on our machine using this tool.

In [4]:
from langchain_community.tools import ShellTool
shell=ShellTool()
response=shell.invoke("ls")
print(response)

Executing command:
 ls
sample_data



/usr/local/lib/python3.12/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


# Custom Tools

## 1. Mutiply function Tool

In [5]:
from langchain_core.tools import tool


In [6]:
#step 1 : create a fun
def mutiply(a,b):
  return a*b

In [7]:
#step 2 :  add type hint
def mutiply(a:int,b:int)->int:
  return a*b

In [8]:
#step 3 : add tool decorator keyword
@tool
def mutiply(a:int,b:int)->int:
  '''Multiply two numbers'''
  return a*b

In [9]:
result=mutiply.invoke({"a":4,"b":5})
print(result)

20


In [10]:
# tool detail
print(mutiply.name)
print(mutiply.description)
print(mutiply.args)

mutiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


What we send to the LLM that is tool scehma and lets see what is the schema of this multiply tool

In [11]:
print(mutiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'mutiply', 'type': 'object'}


# Method 2 -Using Structural Tool

In [12]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel,Field
#

In [13]:
class MultiplyInput(BaseModel):
  a:int=Field(description="first number")
  b:int=Field(description="second number")

In [14]:
def multuliply(a:int,b:int)->int:
  return a*b

In [15]:
multiply_tool=StructuredTool.from_function(
    func=multuliply,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [16]:
result=multiply_tool.invoke({"a":20,"b":5})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

100
multiply
Multiply two numbers
{'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}


# Method 3 : BaseTool with Type

In [17]:
from langchain.tools import BaseTool
from typing import Type

In [18]:
#create schema using pydantic
class MultiplyInput(BaseModel):
  a:int=Field(description="first number")
  b:int=Field(description="second number")

In [21]:
class MuttiplyTool(BaseTool):
  name:str="muitiply"
  description:str="Multiply two number"
  args_schema : Type[BaseModel]=MultiplyInput

  def _run(self,a:int,b:int):
    return a*b

In [22]:
mutiplt_tool=MuttiplyTool()

In [23]:
result=multiply_tool.invoke({'a':2,'b':2})
print(result)

4


# Toolkit
 when we have similar tools related to one domain then we collect them in toolkit.

 Example below


In [24]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [25]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [26]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers
